In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pickle
import pathlib
import os
import copy
import datetime
import time
# sklearn?


In [96]:
data = pd.read_csv("Data/subset-3-sous-ensemble-3.csv", encoding='latin1')

In [97]:
print(data.shape)
print(data.columns)

(1405579, 35)
Index(['LEVEL1ID', 'LEVEL2ID', 'LEVEL3ID', 'LEVEL4ID', 'LEVEL5ID', 'SURVEYR',
       'BYCOND', 'DESCRIP_E', 'DESCRIP_F', 'DEMCODE', 'QUESTION', 'TITLE_E',
       'TITLE_F', 'ANSWER1', 'ANSWER2', 'ANSWER3', 'ANSWER4', 'ANSWER5',
       'ANSWER6', 'ANSWER7', 'MOST_POSITIVE_OR_LEAST_NEGATIVE',
       'NEUTRAL_OR_MIDDLE_CATEGORY', 'MOST_NEGATIVE_OR_LEAST_POSITIVE',
       'AGREE', 'SCORE5', 'SCORE100', 'ANSCOUNT', 'DEPT_E', 'DEPT_F',
       'INDICATORID', 'INDICATORENG', 'INDICATORFRA', 'SUBINDICATORID',
       'SUBINDICATORENG', 'SUBINDICATORFRA'],
      dtype='object')


In [99]:
# drop useless columns
# drop those with only 1 value: LEVEL2ID, LEVEL3ID, LEVEL4ID, LEVEL5ID
empty = []
for col in data.columns:
    uni = data[col].unique()
    if len(uni)<2:
        # print(col, uni)
        data.drop(columns=[col], inplace=True)

# DEMCODE = BYCOND, DESCRIP_E, DESCRIP_F
# QUESTION = TITLE_E, TITLE_F
# INDICATORID = INDICATORENG, INDICATORFRA
# SUBINDICATORID = SUBINDICATORENG, SUBINDICATORFRA
# LEVEL1ID = DEPT_E, DEPT_F
# SCORE5 = X*SCORE100 + Y
useless = ['BYCOND', 'DESCRIP_E', 'DESCRIP_F', 'TITLE_E', 'TITLE_F', 'SUBINDICATORENG', 'SUBINDICATORFRA', 'DEPT_E', 'DEPT_F', 'INDICATORENG', 'INDICATORFRA', 'SCORE100']
try:
    data.drop(columns=useless, inplace=True)
except:
    pass
# drop empty lines
data.drop(data[data['ANSWER1'] == ' '].index, inplace=True)

# set ints
data = data.astype({'SCORE5': 'float'}, copy=False)
data['QUESTION'] = data['QUESTION'].map(lambda x: x.lstrip('Q'))
data = data.astype({col: 'int' for col in ['ANSWER1', 'ANSWER2', 'ANSWER3', 'ANSWER4', 'ANSWER5', 'ANSWER6', 'ANSWER7',
    'MOST_POSITIVE_OR_LEAST_NEGATIVE', 'NEUTRAL_OR_MIDDLE_CATEGORY', 'MOST_NEGATIVE_OR_LEAST_POSITIVE', 'AGREE',
    'ANSCOUNT']}, copy=False)

print(data.shape)
print(data.columns)

(583123, 19)
Index(['LEVEL1ID', 'SURVEYR', 'DEMCODE', 'QUESTION', 'ANSWER1', 'ANSWER2',
       'ANSWER3', 'ANSWER4', 'ANSWER5', 'ANSWER6', 'ANSWER7',
       'MOST_POSITIVE_OR_LEAST_NEGATIVE', 'NEUTRAL_OR_MIDDLE_CATEGORY',
       'MOST_NEGATIVE_OR_LEAST_POSITIVE', 'AGREE', 'SCORE5', 'ANSCOUNT',
       'INDICATORID', 'SUBINDICATORID'],
      dtype='object')


In [100]:
for col in data.columns:
    print(col, '\t', len(data[col].unique()), '\t', type(data[col][0]), '\t', data[col][0])

LEVEL1ID 	 68 	 <class 'numpy.int64'> 	 0
SURVEYR 	 3 	 <class 'numpy.int64'> 	 2020
DEMCODE 	 59 	 <class 'numpy.int64'> 	 2011
QUESTION 	 216 	 <class 'str'> 	 01
ANSWER1 	 101 	 <class 'numpy.int64'> 	 35
ANSWER2 	 101 	 <class 'numpy.int64'> 	 45
ANSWER3 	 87 	 <class 'numpy.int64'> 	 6
ANSWER4 	 71 	 <class 'numpy.int64'> 	 11
ANSWER5 	 82 	 <class 'numpy.int64'> 	 3
ANSWER6 	 74 	 <class 'numpy.int64'> 	 0
ANSWER7 	 65 	 <class 'numpy.int64'> 	 0
MOST_POSITIVE_OR_LEAST_NEGATIVE 	 102 	 <class 'numpy.int64'> 	 81
NEUTRAL_OR_MIDDLE_CATEGORY 	 86 	 <class 'numpy.int64'> 	 6
MOST_NEGATIVE_OR_LEAST_POSITIVE 	 98 	 <class 'numpy.int64'> 	 14
AGREE 	 102 	 <class 'numpy.int64'> 	 81
SCORE5 	 366 	 <class 'numpy.float64'> 	 3.99
ANSCOUNT 	 15718 	 <class 'numpy.int64'> 	 73497
INDICATORID 	 6 	 <class 'numpy.int64'> 	 4
SUBINDICATORID 	 23 	 <class 'numpy.int64'> 	 14


In [102]:
print(data['QUESTION'].unique())

['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18a' '18b' '18c' '18d' '18e' '18f' '18g' '18h' '19' '20'
 '21' '22' '23' '24' '25' '26' '27' '28' '29' '30' '31' '32' '33' '34'
 '35' '36' '37' '38' '39' '40' '41' '42' '43a' '43b' '43c' '43d' '44' '45'
 '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56a' '56b' '56c' '56d'
 '56e' '56f' '56g' '57a' '57b' '57c' '57d' '57e' '57f' '57g' '57h' '57i'
 '57j' '57k' '57l' '57m' '58a' '58b' '58c' '58d' '58e' '58f' '58g' '58h'
 '58i' '59a' '59b' '59c' '59d' '59e' '59f' '59g' '59h' '59i' '59j' '59k'
 '59l' '59m' '59n' '59o' '59p' '60' '61' '62' '63a' '63b' '63c' '63d'
 '63e' '63f' '63g' '64a' '64b' '64c' '64d' '64e' '64f' '64g' '64h' '64i'
 '64j' '64k' '64l' '64m' '64n' '65a' '65b' '65c' '65d' '65e' '65f' '65g'
 '65h' '65i' '66a' '66b' '66c' '66d' '66e' '66f' '66g' '66h' '66i' '66j'
 '66k' '66l' '66m' '66n' '66o' '66p' '67' '68' '69' '70a' '70b' '70c'
 '70d' '70e' '70f' '70g' '70h' '70i' '70j' '70k' '70l' '